In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, DynamoDbDriver, ddb, _tr, Timestamp
from mand.core import RefData, RefDataUpdateEvent, Workbook, PrintMonitor

rawdb = DynamoDbDriver(ddb)
db = ObjectDb(rawdb)

_tr.Clock('Portfolio', db=db).write()

In [2]:
class Book(Workbook):
    
    def books(self):
        return [self]

class PortfolioUpdateEvent(RefDataUpdateEvent):

    @node(stored=True)
    def children(self):
        return []
    
class Portfolio(RefData):

    evCls = PortfolioUpdateEvent
    
    @node
    def clock(self):
        return _tr.Clock.get('Portfolio', db=self.meta.db)
    
    @node
    def children(self):
        evs = self.activeEvents()
        if evs:
            return evs[-1].children()
        else:
            return []
        
    def setChildren(self, children, validTime=None, amends=[]):
        ev = self.evCls(entity=self, amends=amends, children=children, db=self.meta.db)
        ev.write(validTime=validTime)
        return ev
    
    def books(self):
        books = set()
        for c in self.children():
            for b in c.books():
                books.add(b)
        return list(books)

_tr.add(Book)
_tr.add(Portfolio)
_tr.add(PortfolioUpdateEvent)

In [3]:
with PrintMonitor():
    
    with db:
        checking = Book('checking').write()
        savings = Book('savings').write()
        brokerage = Book('brokerage').write()
        misc = Book('misc').write()
        margin = Book('margin').write()
        retirement = Book('401K').write()
        kid1Trust = Book('trust1').write()
        kid2Trust = Book('trust2').write()

        pAll = Portfolio('Family').write()
        pKids = Portfolio('Kids').write()
        pBanking = Portfolio('Banking').write()
        pTrading = Portfolio('Trading').write()
        pDerivs = Portfolio('Derivs').write()

def info():
    c = [ o.meta.name() for o in pAll.children() ]
    b = [ o.meta.name() for o in pAll.books() ]
    print 'All children:', c
    print 'All books   :', b
    print

 Object/write write obj: /Global/Book/checking
   DB write metaobj: /Global/Book/checking
 Object/write write obj: /Global/Book/savings
   DB write metaobj: /Global/Book/savings
 Object/write write obj: /Global/Book/brokerage
   DB write metaobj: /Global/Book/brokerage
 Object/write write obj: /Global/Book/misc
   DB write metaobj: /Global/Book/misc
 Object/write write obj: /Global/Book/margin
   DB write metaobj: /Global/Book/margin
 Object/write write obj: /Global/Book/401K
   DB write metaobj: /Global/Book/401K
 Object/write write obj: /Global/Book/trust1
   DB write metaobj: /Global/Book/trust1
 Object/write write obj: /Global/Book/trust2
   DB write metaobj: /Global/Book/trust2
 Object/write write obj: /Global/Portfolio/Family
   DB write metaobj: /Global/Portfolio/Family
 Object/write write obj: /Global/Portfolio/Kids
   DB write metaobj: /Global/Portfolio/Kids
 Object/write write obj: /Global/Portfolio/Banking
   DB write metaobj: /Global/Portfolio/Banking
 Object/write write ob

In [4]:
with PrintMonitor():
    info()

 GetValue begin ctx: Root, key: Portfolio@1079891d0/Portfolio:children
   GetValue/Calc begin ctx: Root, key: Portfolio@1079891d0/Portfolio:children
adding dep on Portfolio@1079891d0/Portfolio:clock for Portfolio@1079891d0/Portfolio:children
     GetValue begin ctx: Root, key: Portfolio@1079891d0/Portfolio:clock
       GetValue/Calc begin ctx: Root, key: Portfolio@1079891d0/Portfolio:clock
         GetValue/Calc end ctx: Root, key: Portfolio@1079891d0/Portfolio:clock
       Context set value: <mand.clock.Clock..., ctx: Root, key: Portfolio@1079891d0/Portfolio:clock
       GetValue end value: <mand.clock.Clock..., ctx: Root, key: Portfolio@1079891d0/Portfolio:clock
adding dep on Clock@10789ff10/Clock:cutoffs for Portfolio@1079891d0/Portfolio:children
     GetValue begin ctx: Root, key: Clock@10789ff10/Clock:cutoffs
       GetValue/Calc begin ctx: Root, key: Clock@10789ff10/Clock:cutoffs
adding dep on Clock@10789ff10/Clock:parent for Clock@10789ff10/Clock:cutoffs
         GetValue begin 

In [5]:
with PrintMonitor():
    pKids.setChildren([kid1Trust, kid2Trust])
    pBanking.setChildren([checking, savings])
    pTrading.setChildren([brokerage, margin, retirement])
    pDerivs.setChildren([pTrading])
    pAll.setChildren([pBanking, pTrading, pKids])

 GetValue begin ctx: Root, key: PortfolioUpdateEvent@107929dd0/RefDataUpdateEvent:_containers
   GetValue/Calc begin ctx: Root, key: PortfolioUpdateEvent@107929dd0/RefDataUpdateEvent:_containers
adding dep on PortfolioUpdateEvent@107929dd0/Event:_containers for PortfolioUpdateEvent@107929dd0/RefDataUpdateEvent:_containers
     GetValue begin ctx: Root, key: PortfolioUpdateEvent@107929dd0/Event:_containers
       GetValue/Calc begin ctx: Root, key: PortfolioUpdateEvent@107929dd0/Event:_containers
adding dep on PortfolioUpdateEvent@107929dd0/Event:amends for PortfolioUpdateEvent@107929dd0/Event:_containers
         GetValue begin ctx: Root, key: PortfolioUpdateEvent@107929dd0/Event:amends
           GetValue from stored value: [], ctx: Root, key: PortfolioUpdateEvent@107929dd0/Event:amends
         GetValue/Calc end ctx: Root, key: PortfolioUpdateEvent@107929dd0/Event:_containers
       Context set value: set([]), ctx: Root, key: PortfolioUpdateEvent@107929dd0/Event:_containers
       Ge

In [6]:
with PrintMonitor():
    info()
    ts = Timestamp()

 GetValue begin ctx: Root, key: Portfolio@1079891d0/Portfolio:children
   GetValue from ctx value: [], ctx: Root, key: Portfolio@1079891d0/Portfolio:children
 GetValue begin ctx: Root, key: Portfolio@1079891d0/Portfolio:children
   GetValue from ctx value: [], ctx: Root, key: Portfolio@1079891d0/Portfolio:children
All children: []
All books   : []



In [7]:
pDerivs.setChildren([pTrading, misc])

with PrintMonitor():        
    info()

adding dep on PortfolioUpdateEvent@1078e5f90/Event:_containers for PortfolioUpdateEvent@1078e5f90/RefDataUpdateEvent:_containers
adding dep on PortfolioUpdateEvent@1078e5f90/Event:amends for PortfolioUpdateEvent@1078e5f90/Event:_containers
adding dep on PortfolioUpdateEvent@1078e5f90/RefDataUpdateEvent:entity for PortfolioUpdateEvent@1078e5f90/RefDataUpdateEvent:_containers
 GetValue begin ctx: Root, key: Portfolio@1079891d0/Portfolio:children
   GetValue from ctx value: [], ctx: Root, key: Portfolio@1079891d0/Portfolio:children
 GetValue begin ctx: Root, key: Portfolio@1079891d0/Portfolio:children
   GetValue from ctx value: [], ctx: Root, key: Portfolio@1079891d0/Portfolio:children
All children: []
All books   : []

